# Randomness and order {#sec-randomness}

This course is about music and evolution,
specifically about using computational models
to better understand the mechanisms underlying certain evolutionary
processes. In order to do so, we will often run simulations 
to observe and interpret the resulting scenarios against the backdrop
of our modeling assumptions and our domain knowledge.

Simulations are useful because for many situations we 
are not able to provide deterministic mathematical formulae to obtain 
outcomes from inputs. Rather, we incorporate our knowledge about the world
in a computational model and use randomness to include the inherent uncertainty
about the exact outcomes of the simulations.

:::{.callout}
Add section on probability.
:::

But randomness is a difficult concept to capture. 
For the purpose of this book, we will simulate a speficic kind of randomness
by sampling from a certain set of items.
Less technically, we can imagine a bag with a certain number of balls in it,
each having a certain color (multiple balls can have the same color). 
A _random uniform sample with replacement_ then corresponds to picking a 
ball from the bag without looking in it and putting it back in the bag.

## Random draws from a bag

Let's try this in Python. We will use the `random` modul of the _NumPy_ library:


In [ ]:
import numpy as np
rng = np.random.default_rng() # initialize a default random generator

bag = range(4)
ball = rng.choice(bag)

print(ball)

If this draw were really random, we would expect that each number is equally likely.
We can test this by repeating this procedure again and again, tallying the result each time.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

samples = []

for i in range(1000):
    ball = rng.choice(bag)
    samples.append(ball)

s = pd.Series(samples)

print(s.head(10))
print(s.value_counts().sort_index())

The number are not exactly the same but pretty close. If we would continue sampling
from our bag, they would get more and more similar to one another.
It is easier to understand this by visualizing it.


In [ ]:
s.value_counts().sort_index().plot(kind="bar")
plt.show()

Now, what if the numbers in the bag were not just numbered,
but had different colors? Let's assume we have another bag, `bag2`,
with 4 balls, three brown, one blue:


In [ ]:
bag2 = [ "blue", "blue", "blue", "brown" ]

samples2 = []

for i in range(1000):
    ball = np.random.choice(bag2)
    samples2.append(ball)

s2 = pd.Series(samples2)

print(s2.head(10))
print(s2.value_counts().sort_index())

s2.value_counts().sort_index().plot(kind="bar")
plt.show()

This is remarkable: by randomly (uniformly) drawing from the 
second bag, the frequencies of all samples approach the ratio of brown to blue
balls (3:1)!

## Composing random melodies

Since this book is about music, let's see how we can use randomness 
to create (a resemblance of) music. For instance, we can 'compose'
a random melody by using only the white keys on a piano within some octave:


In [ ]:
notes = list("CDEFGAB")
melody = rng.choice(notes, size=10)
print(melody, end=" ")

We composed a little melody by randomly drawing a note from the list of notes.
This is also called _sampling_. Note that some notes repeat, showing that we 
sample with replacement: after each draw, the note is put back in the bag,
so to speak. Of course, there are many things that we would have to generate, too, to make this a real melody. For instance, we do not know the duration of 
any of these notes, we don't know the meter nor the key, we don't know the
tempo or volume, and so on. But our goal here is not to create a beautiful piece 
of music, but rather to show how we can use randomness to generate something.

As you might remember from the previous chapter, we can also write a function 
to do this, so that we can perform this operation (composition of a random 
melody) more easily, while at the same time having more control over it 
through its parameters. The following function does exactly this, having 
only one parameter that controlls the length of the melody (the number of
notes to be sampled).


In [ ]:
def melody(n):
    notes = list("CDEFGAB")
    return rng.choice(notes, size=n)

We can now use this function to easily create random melodies of 
different lengths:


In [ ]:
print(melody(7))

In [ ]:
print(melody(12))

## Synthesizing a corpus

The functionalities introduced above allow us to synthesize an artificial corpus
of melodies, here simplified as lists of pitch classes and containing varying
numbers of notes.


In [ ]:
N = 4 # number of pieces in the corpus
corpus = [ melody(12) for _ in range(N)]

The first three melodies of our corpus are:


In [ ]:
for mel in corpus:
    print(mel)

Of course, melodies are not always of the same length.
We could vary the lenght of the melodies by creating a hand-crafted list
specifying the number of notes for each melody in the corpus.


In [ ]:
corpus = [ melody(n) for n in [10, 5, 7, 13] ]

for mel in corpus:
    print(mel)

However, specifying the lenghts of the melodies for a large corpus
would be a very time-consuming task. In order to model the 
variability in length of melodies in a musical corpus, we will 
_randomly sample_ them from a specified probability distribution.
A good candidate for such a distribution is the [Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution),
that we can access from our random number generator `rng`.


In [ ]:
#| label: fig-piece-lengths
#| fig-cap: Distribution of melody lengths in the corpus."

lam = 25 # average number of notes in melody
N = 1000 # number of pieces in the corpus 

corpus = [ melody(rng.poisson(lam=lam)) for _ in range(N) ]

lengths = pd.Series([ len(m) for m in corpus ]).value_counts()

idx = range(0, max(lengths))
lengths = lengths.sort_index().reindex(idx).fillna(0)
plt.bar(idx, lengths)
plt.axvline(lam, c="red")
plt.show()

Now the variable `corpus` contains lists of pitch classes (_aka_ melodies)
of different lengths, most of them around the preset average value `lam`,[^1]
also indicated by the vertical red line.
It is moreover evident that the corpus contains rather few very short or long melodies.

[^1]: Short for the Greek letter $\lambda$ ("lambda").

We can, of course, not only observe the distribution of melody lengths, but also 
look at the overall distribution of note occurrence in the corpus:


In [ ]:
#| label: fig-note-freqs
#| fig-cap: Overall note frequencies in the artificial corpus.

counts = []

for m in corpus:
    c = pd.Series(m)
    counts.append(c)

pd.concat(counts).value_counts().plot(kind="bar")
plt.show()

At this point, we should stop and celebrate. We have just written our first _probabilistic model_
to generate melodies. Admittedly, it is not a very good model for actual melodies, for example
because notes are drawn _uniformly at random_ from the set of diatonic pitch classes
using the `.choice()` method, which leads to the somewhat unrealistic picture in @fig-note-freqs.
One would expect that in real melodies some notes occur more often than others
and that the occurrence of a note does, for instance, also depend on the notes that come before and after it.
But, in principle, these other constraints could be added to our model to make it more realistic.
The point here was mainly to illustrate how artificial corpora can be generated probabilistically.
This will prove useful later on because it allows us to compare real-world corpora of music against 
synthetic ones generated by our models.

::: {.callout-tip}
## Exercise

Expand our melody model so that it also includes octave information for each pitch class
in order to make it a bit more musical.
:::

## Pattern search

### Incipits 

Now that we have a corpus that we understand very well because we specified 
how it has been created, we can apply some simple analytical questions in order 
to warm up for later. For instance, we could want to have a function that 
allows us to search for _incipits_. Incipits are the beginnings of musical melodies
that already to characterize themes and motives because incipits are often characteristic. 
For example, we would want to look for all melodies that begin with "C", "D", "E" and, 
for simplicity, we might want to pass a string like "CDE" to the function to facilitate the input.


In [ ]:
import re 

def find_incipit(incip="", mel=None):
    melody = "".join(mel)
    if re.search("^" + incip, melody):
        return True
    else:
        return False

for m in corpus[:10]:
    if find_incipit(incip="CDE", mel=m):
        print("".join(m))

### Finals

We can apply a similar logic to find finals, the 
last notes of a melody. Instead of only allowing
to search for a single note as a final, we will 
allow more generally to allow for a pattern that concludes
a melody:


In [ ]:
def find_finals(end="", mel=None):
    melody = "".join(mel)
    if re.search(end + "$", melody):
        return True 

for m in corpus[:100]:
    if find_finals(end="GC", mel=m):
        print(m)

As you can see, all found melodies end with a falling perfect
fifth form "G" to "C".

The last function, `find_finals()`, 
introduced the "^" (caret) character.
In the context of regular expressions, this character signifies 
"at the end of a string", exactly what we needed to find finals.

### Patterns more generally

::: {.callout-warning}
## Todo
Introduce regexes more flexibly and write a general pattern matcher.
:::

## Horrible Homophony

Four-part writing is a core part of Western composition history.
Here, we will build a mock version of a four-part chorale by randomly 
generating each voice and putting them together in a table. 
Doing so will show you how you can create tables, which we will need later on.
The most popular way to work with tables in Python is by using the `pandas` library. 
In `pandas`, tables are called 'data frames', and there is a `DataFrame`
object to represent tables. Let's see how we could create a four-part homophonic 
chorale with eight 'chords':


In [ ]:
import pandas as pd

n = 8

chorale = pd.DataFrame({
    "S" : melody(n),
    "A" : melody(n),
    "T" : melody(n),
    "B" : melody(n)
})

The variable `chorale` now stores our little composition and we can inspect it:


In [ ]:
chorale

Here we have generated each voice using the `melody` function. 
We can use it to create a new function that will directly give us a new 
chorale of a certain length:


In [ ]:
def chorale(n):
    df = pd.DataFrame({
        "S" : melody(n=n),
        "A" : melody(n=n),
        "T" : melody(n=n),
        "B" : melody(n=n)
    })

    return df

In [ ]:
my_chorale = chorale(n=12)
my_chorale

It will look a bit closer to musical notation if 
we transpose the data frame by using the `.transpose()` method:


In [ ]:
my_chorale.transpose()

## Accessing data

Having the variable `my_chorale` store our data frame, 
this is how we can access individual voices:


In [ ]:
my_chorale["T"]

You can verify that it is the same 'melody' as above in the chorale. 
If we want a specific note from this voice, say the fifth one, 
we can access is this way:


In [ ]:
my_chorale["T"][4]

We first select the "T" column, and then select the fifth element 
(remember that we start counting at 0, so we need to insert 4 to get the 
fifth). We can also get entire ranges of a voice:


In [ ]:
my_chorale["A"][4:8]

This gives us the fifths to ninth note in the Alto voice. 
If we want to apply the same logic also to column ranges, 
we have to write it a bit differently using the `.loc()`
method for localising data:


In [ ]:
my_chorale.loc[1:3, "S":"A"]

`.loc()` takes two arguments: the rows (or row range),
and the columns (or column range). We can use it to 'slice'
our data frame in order to get specific portions of it.